In [6]:
! pip install tweepy==4.8.0

     |████████████████████████████████| 77 kB 1.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install loguru

     |████████████████████████████████| 58 kB 2.8 MB/s 


In [1]:
import os
import tweepy
import pandas
from loguru import logger
from sys import stderr

consumer_key = 'Bkzg44MHgtQszMvCdFhJEindf'
consumer_secret = '07vhPzTsz2r6QTy3TzISuDN0NJZNLt6SMBJIcGpbIFEZunwsGw'
access_token_key = '1512683317820239876-KphHJoncgY9fJpr5yw8WvEJdRYXZzF'
access_token_secret = 'gnUcKp6V7KrkUTLJbeTs5BCkiinevc1jkt181djFhUVmW'

bearer_token = "AAAAAAAAAAAAAAAAAAAAAKMxbQEAAAAAXL9RA3fotZaAejPj5lYjvdm9DaE%3DQ7BpU1w6ZxBBzEiOaKqBXA1lIuUISRptYlxAC6SaOBOA9Drfw9"

In [2]:
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

In [42]:
loglevel = os.getenv("LOGLEVEL", "INFO").upper()
logger.remove()
logger.add(stderr, level=loglevel)

collected_tweets = []
total_collected = 0

max_results = 100

for i, tweets in enumerate(tweepy.Paginator(
      client.search_recent_tweets,
      query = "Melbourne lang:en",
      tweet_fields = [
          'context_annotations',
          'created_at',
          'geo',
          'lang',
          'conversation_id',
          'public_metrics',
          'referenced_tweets',
          'in_reply_to_user_id',
      ],
      user_fields = [
          'profile_image_url',
          'name',
          'username',
          'created_at',
          'description',
          'entities',
          'location',
          'protected',
          'public_metrics',
          'verified',
      ],
      media_fields = [
          "media_key",
          "type",
          "duration_ms",
          "public_metrics",
          "width",
          "height",
          "preview_image_url",
          "alt_text"
      ],
      expansions = [
          "author_id",
          "referenced_tweets.id",
          "referenced_tweets.id.author_id",
          "attachments.media_keys",
      ],
      max_results=max_results
  )):
  max_results = max_results
  logger.info(f"Retrieved page {i+1} -- Collected {total_collected} tweets so far.")
  if not (tweets.includes.get('users')):
    break
  users = {u["id"]: u for u in tweets.includes['users']}
  media = {media["media_key"]: media for media in tweets.includes.get("media")} if tweets.includes.get("media") else None
  for j, tweet in enumerate(tweets.data):
    new_row = {}
        
    user = users[tweet.author_id]

    # get media
    attachments = tweet['attachments']
    media_keys = attachments.get('media_keys') if attachments else None
    tweet_media = media.get(media_keys[0] if media_keys else media_keys) if media else None

    new_row = {
                "tweet_text": tweet.text,
                "tweet_id": tweet.id,
                "tweet_created_at": tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"),
                "tweet_public_metrics": tweet.public_metrics,
                "tweet_in_reply_to_user_id": tweet.in_reply_to_user_id,
                "tweet_conversation_id": tweet.conversation_id,
                "tweet_lang": tweet.lang,
                "tweet_context_annotations": tweet.context_annotations,
                "tweet_geo": tweet.geo,

                "author_id": user.id,
                "author_name": user.name,
                "author_username": user.username,
                "author_created_at": user.created_at.strftime("%Y-%m-%d %H:%M:%S"),
                "author_description": user.description,
                "author_entities": user.entities,
                "author_location": user.location,
                "author_is_protected": user.protected,
                "author_is_verified": user.verified,
                "author_profile_image_url": user.profile_image_url,
                "author_public_metrics": user.public_metrics,

                "media_key": tweet_media.get("media_key") if tweet_media else None,
                "media_type": tweet_media.get("media_type") if tweet_media else None,
                "media_duration_ms": tweet_media.get("duration_ms") if tweet_media else None,
                "media_height": tweet_media.get("height") if tweet_media else None,
                "media_width": tweet_media.get("width") if tweet_media else None,
                "media_preview_image_url": tweet_media.get("preview_image_url") if tweet_media else None,
                "media_alt_text": tweet_media.get("alt_text") if tweet_media else None,
                "media_view_count": (
                    tweet_media.get("public_metrics").get("view_count")
                        if tweet_media and 
                            tweet_media.get("public_metrics") and
                            tweet_media["public_metrics"].get("view_count")
                        else None
                )
            }

    collected_tweets.append(new_row)
    total_collected += 1


2022-04-24 10:52:05.712 | INFO     | __main__:<module>:54 - Retrieved page 1 -- Collected 0 tweets so far.
2022-04-24 10:52:06.474 | INFO     | __main__:<module>:54 - Retrieved page 2 -- Collected 100 tweets so far.
2022-04-24 10:52:07.049 | INFO     | __main__:<module>:54 - Retrieved page 3 -- Collected 200 tweets so far.
2022-04-24 10:52:07.632 | INFO     | __main__:<module>:54 - Retrieved page 4 -- Collected 300 tweets so far.


KeyboardInterrupt: ignored

In [43]:
print(collected_tweets[0].keys())
for tweet in collected_tweets:
  print(tweet['tweet_geo'])

dict_keys(['tweet_text', 'tweet_id', 'tweet_created_at', 'tweet_public_metrics', 'tweet_in_reply_to_user_id', 'tweet_conversation_id', 'tweet_lang', 'tweet_context_annotations', 'tweet_geo', 'author_id', 'author_name', 'author_username', 'author_created_at', 'author_description', 'author_entities', 'author_location', 'author_is_protected', 'author_is_verified', 'author_profile_image_url', 'author_public_metrics', 'media_key', 'media_type', 'media_duration_ms', 'media_height', 'media_width', 'media_preview_image_url', 'media_alt_text', 'media_view_count'])
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
{'place_id': '01864a8a64df9dc4'}
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [46]:
type(collected_tweets)
collected_tweets[0]

{'author_created_at': '2013-04-04 08:12:51',
 'author_description': '',
 'author_entities': None,
 'author_id': 1326437888,
 'author_is_protected': False,
 'author_is_verified': False,
 'author_location': 'Geelong',
 'author_name': 'Maree Avery',
 'author_profile_image_url': 'https://pbs.twimg.com/profile_images/3626035769/0d57280776853fbe5f8282c8af98fa0f_normal.jpeg',
 'author_public_metrics': {'followers_count': 188,
  'following_count': 475,
  'listed_count': 0,
  'tweet_count': 2391},
 'author_username': 'mareeave',
 'media_alt_text': None,
 'media_duration_ms': None,
 'media_height': None,
 'media_key': None,
 'media_preview_image_url': None,
 'media_type': None,
 'media_view_count': None,
 'media_width': None,
 'tweet_context_annotations': [],
 'tweet_conversation_id': 1518180921966534656,
 'tweet_created_at': '2022-04-24 10:51:52',
 'tweet_geo': None,
 'tweet_id': 1518180921966534656,
 'tweet_in_reply_to_user_id': None,
 'tweet_lang': 'en',
 'tweet_public_metrics': {'like_count'